In [ ]:
#exercise_grph

# Read the input in FASTA format
def read_fasta(file_path):
    with open(file_path, 'r') as file:
        sequences = []       #list to store sequences as tuples of (header, sequence)
        header = None        #variable to store the current header
        sequence = []        #list to accumulate sequence lines
        
        # Read through each line in the file
        for line in file:
            line = line.strip()             #remove leading/trailing whitespace
            if line.startswith(">"):        #if the line is a header (starts with ">")
                if header is not None:      #if there is a previous sequence, store it
                    sequences.append((header, ''.join(sequence)))  # join sequence parts and append
                header = line[1:]           #remove the ">" character from the header
                sequence = []               #reset the sequence list for the new header
            else:
                sequence.append(line)       #add sequence lines to the current sequence list
    
    return sequences

def overlap_graph(sequences, k):
    adjacency_list = []
    
    #compare all pairs of sequences
    for i in range(len(sequences)):
        for j in range(len(sequences)):
            if i != j:                          #do not compare a string with itself
                header_i, seq_i = sequences[i]
                header_j, seq_j = sequences[j]
                
                #check if suffix of length k from seq_i matches prefix of length k from seq_j
                if seq_i[-k:] == seq_j[:k]:
                    adjacency_list.append((header_i, header_j))
    
    return adjacency_list

def main():
    file_path = 'rosalind_grph.txt'  
    k = 3  #length of the suffix and prefix we are looking for
    
    sequences = read_fasta(file_path)
    
    edges = overlap_graph(sequences, k)
    
    for edge in edges:
        print(f"{edge[0]} {edge[1]}")

if __name__ == "__main__":
    main()


In [ ]:
#exercise_tree

def dfs(node, adj, visited):
    #depth-First Search (DFS) to explore all nodes in the current connected component
    stack = [node]                         #initialize stack for DFS
    while stack:
        current = stack.pop()              #op a node from the stack to explore
        for neighbor in adj[current]:      #iterate over all neighbors of the current node
            if not visited[neighbor]:      #if the neighbor hasn't been visited...
                visited[neighbor] = True   #...mark the neighbor as visited
                stack.append(neighbor)     #add the neighbor to the stack 

def read_input(file_path):
    #read the graph input from the file and return the number of nodes and adjacency list
    with open(file_path, 'r') as file:
        n = int(file.readline().strip())  #read the number of nodes
        
        #initialize the adjacency list (dictionary with empty lists)
        adj = {i: [] for i in range(1, n+1)}
        
        #read the edges and populate the adjacency list
        for line in file:
            u, v = map(int, line.strip().split())  #read an edge (u, v)
            adj[u].append(v)  # add v to the adjacency list of u
            adj[v].append(u)  # add u to the adjacency list of v 
    
    return n, adj 

def main():
    file_path = 'rosalind_tree.txt'  
    n, adj = read_input(file_path)  
    
    #array to keep track of visited nodes (initially set to False for all)
    visited = [False] * (n + 1)
    
    #variable to count the number of connected components
    components = 0
    for i in range(1, n+1):  #loop through all nodes
        if not visited[i]: 
            visited[i] = True
            dfs(i, adj, visited)
            components += 1  #increment the count of connected components
    
    print(components - 1)  

if __name__ == "__main__":
    main()  # Run the main function


In [ ]:
#exercise_long

from Bio import SeqIO

def _get_overlap_strings(s1, s2):
    #This function finds the best overlap between two strings s1 and s2
    combine_strings = []  #To store combined strings after overlap
    overlap_strings = []  #To store the actual overlap strings

    #check for overlap where suffix of s1 matches prefix of s2
    for i in range(len(s1)):
        if s1[i:] == s2[:len(s1) - i]:
            overlap_strings.append(s1[i:])  # store the overlapping part
            combine_strings.append(s1 + s2[len(s1) - i:])  #combine s1 and s2 with overlap
            break

    #check for overlap where suffix of s2 matches prefix of s1
    for i in range(len(s2)):
        if s2[i:] == s1[:len(s2) - i]:
            overlap_strings.append(s2[i:])  #store the overlapping part
            combine_strings.append(s2 + s1[len(s2) - i:])  #combine s2 and s1 with overlap
            break

    #if no overlap found, return empty strings
    if not overlap_strings:
        return "", ""

    #return the shortest combined string and the longest overlap string
    combine_string = min(combine_strings, key=len)  # shortest combination
    overlap_string = max(overlap_strings, key=len)  # longest overlap
    return combine_string, overlap_string

def find_superstring(strings):
    #this function finds the shortest superstring by merging overlapping strings
    temp = strings  #copy the input strings to a temporary list

    #keep merging the most overlapping strings until one string remains
    while len(temp) > 1:
        most_overlapping_string = ""         #to store the best combination
        most_overlapping_string_pair = []    #to store the pair of strings with best overlap
        most_overlapping_string_length = 0   #to track the length of the best overlap

        #compare each pair of strings to find the best overlap
        for i in range(len(temp) - 1):
            for j in range(i + 1, len(temp)):
                combine_string, overlap_string = _get_overlap_strings(temp[i], temp[j])
                if len(overlap_string) > most_overlapping_string_length:
                    most_overlapping_string = combine_string             #update the best combination
                    most_overlapping_string_pair = [temp[i], temp[j]]    #update the pair
                    most_overlapping_string_length = len(overlap_string) #update the length of overlap

        #remove the two strings with the best overlap and add the combined string
        temp.remove(most_overlapping_string_pair[0])
        temp.remove(most_overlapping_string_pair[1])
        temp.append(most_overlapping_string)  #add the newly combined string to the list

    return temp[0]

def main(input_file):
    seq_strings = []  # List to store the sequencest
    with open(input_file, 'r') as fa:
        for seq_record in SeqIO.parse(fa, 'fasta'):
            seq_strings.append(str(seq_record.seq))  

    #find the shortest superstring that contains all the given sequences
    shortest_superstring = find_superstring(seq_strings)
    print(shortest_superstring)

if __name__ == "__main__":
    main("rosalind_long.txt")


In [ ]:
#exercise_deg
def calculate_degrees(num_vertices, edges):
    degrees = [0] * num_vertices                            #will store the degree of each vertex
    
    #for each edge, increment the degree of the involved vertices 
    for u, v in edges:
        degrees[u - 1] += 1                                #decrement by 1 to work with 0-based index
        degrees[v - 1] += 1                                #decrement by 1 to work with 0-based index

    return degrees

def read_input(file_path):
    #read the input data from a file
    with open(file_path, 'r') as file:
        num_vertices, num_edges  = map(int, file.readline().split())  #read num_vertices and num_edges 
        edges = []
        
        #read the edges
        for _ in range(num_edges ):
            u, v = map(int, file.readline().split())
            edges.append((u, v))
    
    return num_vertices, edges

def main():
    #read input from a file
    file_path = 'rosalind_deg.txt'
    num_vertices, edges = read_input(file_path)
    
    #compute the degree of each vertex
    degrees = calculate_degrees(num_vertices, edges)
    
    print(' '.join(map(str, degrees)))

if __name__ == "__main__":
    main()


In [ ]:
#exercise_ddeg
def compute_degree_sum(vertex_count, edges):
    #initialize degree array
    degree = [0] * vertex_count                          #will store the degree of vertex i+1
    adjacency_list  = [[] for _ in range(vertex_count)]  #adjacency list for the graph
    
    #populate adjacency list and degree array
    for u, v in edges:
        adjacency_list [u - 1].append(v - 1)             #0-based index
        adjacency_list [v - 1].append(u - 1)             #0-based index
        degree[u - 1] += 1                               #increment degree of vertex u
        degree[v - 1] += 1                               #increment degree of vertex v
    
    #calculate the sum of degrees of neighbors for each vertex
    degree_sum = [0] * vertex_count                      #will store the sum of the degrees of neighbors
    for i in range(vertex_count):
        tot_degrees = 0
        for neighbor in adjacency_list [i]:
            tot_degrees += degree[neighbor]              #add the degree of each neighbor
        degree_sum[i] = tot_degrees
    
    return degree_sum

def read_input(file_path):
    #read the input data from a file
    with open(file_path, 'r') as file:
        vertex_count, edge_count  = map(int, file.readline().split())  #read vertex_count and medge_count 
        edges = []
        
        #read the edges
        for _ in range(edge_count ):
            u, v = map(int, file.readline().split())
            edges.append((u, v))
    
    return vertex_count, edges

def main():
    #read input from a file
    file_path = 'rosalind_ddeg.txt' 
    n, edges = read_input(file_path)
    
    #cmpute the degree of each vertex
    degrees = compute_degree_sum(n, edges)
    
    print(' '.join(map(str, degrees)))

if __name__ == "__main__":
    main()


In [ ]:
#exercise_cc

def depth_first_search(graph, visited_nodes, start_node):
    #perform depth_first_search to visit all nodes in the component starting from 'start_node'
    node_stack  = [start_node]                           #initialize node_stack with the start node
    while node_stack :
        current = node_stack.pop()                       #the .pop() method removes and returns the last element from a list
        if not visited_nodes[current]:                   #if the current node hasn't been visited...
            visited_nodes[current] = True                #...mark it as True
            for neighbor in graph[current]:              #visit all neighbors of the current node
                if not visited_nodes[neighbor]:          #if the neighbor hasn't been visited...
                    node_stack.append(neighbor)          #...add neighbor to stack

#count the number of connected components in the graph
def count_connected_components(vertex_count, edges):
    #build the adjacency list
    graph = [[] for _ in range(vertex_count)]
    for u, v in edges:
        graph[u - 1].append(v - 1)                      # Convert to 0-based index
        graph[v - 1].append(u - 1)                      # Convert to 0-based index
    
    #initialize visited list
    visited_nodes = [False] * vertex_count
    tot_components = 0
    
    #use depth_first_search to explore the graph
    for vertex in range(vertex_count):
        if not visited_nodes[vertex]:
            # If 'vertex' has not been visited yet, we have found a new component
            depth_first_search(graph, visited_nodes, vertex)
            tot_components += 1
    
    #return the number of connected components
    return tot_components

def read_input(file_path):
    #read the input data from a file
    with open(file_path, 'r') as file:
        vertex_count, edge_count  = map(int, file.readline().split())  # Read vertex_count and edge_count
        edges = []
        
        # Read the edges
        for _ in range(edge_count ):
            u, v = map(int, file.readline().split())
            edges.append((u, v))
    
    return vertex_count, edges

def main():
    #read input from a file
    file_path = 'rosalind_cc.txt'
    vertex_count, edges = read_input(file_path)
    
    #compute the number of connected components
    result = count_connected_components(vertex_count, edges)
    
    print(result)

if __name__ == "__main__":
    main()


In [ ]:
#exercise_bfs

def bfs_shortest_paths(n, edges):
    adj_list  = [[] for _ in range(n)]                  #create an empty adjacency list for each node
    for u, v in edges:
        adj_list [u - 1].append(v - 1)                  #convert to 0-based indexing
    
    distances  = [-1] * n                               #distance to all vertices is initially -1
    distances [0] = 0                                   #distance from node 1 (index 0) to itself is 0
    
    for layer in range(n):
        #for each node, check if it's already visited
        for current in range(n):
            if distances [current] == layer:
                #for each unvisited neighbor of the current node, update the distance
                for neighbor in adj_list [current]:
                    if distances [neighbor] == -1:        #if neighbor hasn't been visited...
                        distances [neighbor] = layer + 1  #...set the distance to this neighbor
    
    return distances 

def read_input(file_path):
    #read the input data from a file
    with open(file_path, 'r') as file:
        n, m = map(int, file.readline().split())          #read the number of nodes (n) and edges (m)
        edges = []
        
        #read the edges and append it to the list
        for _ in range(m):
            u, v = map(int, file.readline().split())
            edges.append((u, v))
    
    return n, edges

def main():
    file_path = 'rosalind_bfs.txt' 
    n, edges = read_input(file_path)
    
    distances = bfs_shortest_paths(n, edges)
    
    print(' '.join(map(str, distances)))

if __name__ == "__main__":
    main()


In [ ]:
#exercise_dij

import heapq

def dijkstra(num_nodes, edges):
    #build the adjacency list
    adj_list = [[] for _ in range(num_nodes)]
    for u, v, w in edges:
        adj_list[u - 1].append((v - 1, w))
    
    #initialize distances and priority queue
    INF = float('inf')
    distances = [INF] * num_nodes
    distances[0] = 0  
    priority_queue = [(0, 0)]
    
    #dijkstra's algorithm using a priority queue
    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)
        
        #skip if the current distance is already larger than the known shortest distance
        if current_distance > distances[current_vertex]:
            continue
        
        #update distances to neighbors
        for neighbor, weight in adj_list[current_vertex]:
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
    
    #convert unreachable vertices to -1
    for i in range(num_nodes):
        if distances[i] == INF:
            distances[i] = -1
    
    return distances

def read_input(file_path):
    #read the input data from a file
    with open(file_path, 'r') as file:
        num_nodes, num_edges  = map(int, file.readline().split()) 
        edges = []
        
        #read the edges
        for _ in range(num_edges ):
            u, v, w = map(int, file.readline().split())
            edges.append((u, v, w))
    
    return num_nodes, edges

def main():
    file_path = 'rosalind_dij.txt' 
    num_nodes, edges = read_input(file_path)
    
    distances = dijkstra(num_nodes, edges)
    
    print(' '.join(map(str, distances)))

if __name__ == "__main__":
    main()


In [ ]:
#exercise_dag

import networkx as nx

def check_acyclicity(input_file):
    results = []                                               #store the results for each graph
    with open(input_file, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]   #read and clean lines from the file
    
    num_graphs  = int(lines[0])                                #first line contains the number of graphs
    index = 1                                                  #initialize the line index to read the subsequent data

    for _ in range(num_graphs):
        #read number of vertices and edges
        num_vertices, m = map(int, lines[index].split())
        index += 1
   
        #read the edges of the graph
        edges = [tuple(map(int, line.split())) for line in lines[index:index + m]]
        index += m

        #create a directed graph and add the edges
        graph  = nx.DiGraph()
        graph.add_edges_from(edges)
 
        #check if the graph is acyclic and store the result
        if nx.is_directed_acyclic_graph(graph):
            results.append("1")
        else:
            results.append("-1")

    return results

def main(input_file):
        results = check_acyclicity(input_file)
        print(" ".join(results))

if __name__ == "__main__":

    main ('rosalind_dag.txt')

In [ ]:
#exercise_bip

from collections import deque

#function to read input data from a file
def read_input(file_path):
    with open(file_path, 'r') as file:
        k = int(file.readline().strip())               #number of graphs
        graphs = []
        
        #read each graph
        for _ in range(k):
          #read a line for number of vertices and edges (n, m)
            line = file.readline().strip()
            #skip empty lines, if any
            while not line:
                line = file.readline().strip()
            
            n, m = map(int, line.split())             #number of vertices and edges
            edges = []
            
            for _ in range(m):
                line = file.readline().strip()
                #skip empty lines for edges
                while not line:
                    line = file.readline().strip()
                u, v = map(int, line.split())
                edges.append((u, v))                  #store edges as 1-based indexing
            
            graphs.append((n, edges))
    
    return graphs

#function to check if a graph is bipartite using BFS
def is_bipartite(n, edges):
    #create an adjacency list from the edge list
    adj = [[] for _ in range(n + 1)]                  #using 1-based indexing
    for u, v in edges:
        adj[u].append(v)
        adj[v].append(u)
    
    color = [0] * (n + 1)                             #color array, 0 = uncolored, 1 and 2 will represent the two colors
    
    #BFS function to check bipartiteness
    def bfs(start):
        queue = deque([start])
        color[start] = 1                              #start coloring with 1
        
        while queue:
            node = queue.popleft()
            current_color = color[node]
            
            for neighbor in adj[node]:
                if color[neighbor] == 0:              #if uncolored, color it with opposite color
                    color[neighbor] = 3 - current_color  #toggle between 1 and 2
                    queue.append(neighbor)
                elif color[neighbor] == current_color:  #if the neighbor has the same color, not bipartite
                    return False
        return True
    
    #check all disconnected components
    for i in range(1, n + 1):
        if color[i] == 0:                            #if the vertex is not colored, start a BFS from it
            if not bfs(i):
                return -1                            #return -1 if any component is not bipartite
    return 1                                         #if all components are bipartite


def main():
    file_path = 'rosalind_bip.txt'   
    graphs = read_input(file_path) 
    
    results = []
    for n, edges in graphs:
        result = is_bipartite(n, edges)            #check if each graph is bipartite
        results.append(result)
    
    print(" ".join(map(str, results)))

if __name__ == "__main__":
    main()


In [ ]:
#exercise_bf

import math

def bellman_ford(n, edges):
    distances  = [math.inf] * n                #initialize the distance array with infinity and distance to the source as 0
    distances [0] = 0                          #distance from the source vertex (1) to itself is 0
    
    #prform Bellman-Ford algorithm for n-1 iterations
    for _ in range(n - 1):
        for u, v, w in edges:
            if distances [u] != math.inf and distances [u] + w < distances [v]:
                distances [v] = distances [u] + w
    
    result = ['x' if d == math.inf else d for d in distances]    #replace infinity with 'x' for unreachable vertices
    
    return result

def read_input(file_path):
    with open(file_path, 'r') as file:
        n, m = map(int, file.readline().split())  #read n (vertices) and m (edges)
        edges = []
        
        for _ in range(m):
            u, v, w = map(int, file.readline().split())  #read each edge (u, v, w)
            edges.append((u - 1, v - 1, w))              #store edges with 0-based indexing
        
    return n, edges

def main():
    file_path = 'rosalind_bf.txt'
    n, edges = read_input(file_path)
    
    result = bellman_ford(n, edges)
    
    print(' '.join(map(str, result)))

if __name__ == "__main__":
    main()


In [ ]:
#exercise_cte

import heapq
from collections import defaultdict

def find_shortest_cycle(k, graph_data):
    result = []                                    #list to store the result for each graph
    
    #iterate through each graph
    for graph in graph_data:
        num_vertices, num_edges, edges = graph
        first_edge = edges[0]                      #get the first edge 
        start, end, weight = first_edge            #extract the start, end, and weight of the first edge

        #build the adjacency list for the graph
        adj_list = defaultdict(list)
        for u, v, w in edges:
            adj_list[u].append((v, w))             #add each edge to the adjacency list

        adj_list[start] = [(node, w) for node, w in adj_list[start] if node != end]  #temporarily remove the first edge

        #tunction to perform Dijkstra's algorithm and find the shortest path from 'source' to 'target'
        def dijkstra(source, target):
            distances = {i: float('inf') for i in range(1, num_vertices + 1)}   #initialize distances for all nodes as infinity
            distances[source] = 0                                               #distance to the source node is 0
            pq = [(0, source)]                                                  #priority queue to store nodes

            #process the priority queue
            while pq:
                curr_dist, curr_node = heapq.heappop(pq)                        #pop the node with the smallest distance

                if curr_node == target:
                    return curr_dist                                           #return the shortest distance to the target node

                if curr_dist > distances[curr_node]:                           #skip if we already found a shorter path
                    continue

                #explore the neighbors of the current node
                for neighbor, edge_weight in adj_list[curr_node]:
                    new_dist = curr_dist + edge_weight                        #calculate the new distance for the neighbor
                    if new_dist < distances[neighbor]:                        #if the new distance is shorter, update it
                        distances[neighbor] = new_dist
                        heapq.heappush(pq, (new_dist, neighbor))              #add the neighbor to the priority queue

            return float('inf') 

        shortest_path = dijkstra(end, start)                                  # find the shortest path from 'end' to 'start'

        #if a path exists, the cycle length is the shortest path + the weight of the first edge
        if shortest_path < float('inf'):
            result.append(shortest_path + weight)
        else:
            result.append(-1)                          #if no path is found, return -1 indicating no cycle

    return result

#function to read and parse the input from a file
def read_graph_input(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()              #read all lines from the file

    num_graphs = int(lines[0].strip())        #the first line contains the number of graphs
    graph_data = []                           #list to store the data for all graphs
    line_index = 1                            #initialize the line index to read the graph data

    #iterate through each graph in the input file
    for _ in range(num_graphs):
        #skip any empty lines before reading graph data
        while line_index < len(lines) and not lines[line_index].strip():
            line_index += 1

        if line_index >= len(lines):  #prevent going beyond the file content
            raise ValueError("Unexpected end of file. Not enough data for graphs.")

        #read the number of vertices (n) and edges (m) for the current graph
        try:
            num_vertices, num_edges = map(int, lines[line_index].strip().split())
        except ValueError:
            raise ValueError(f"Malformed line while reading the number of vertices and edges: {lines[line_index]}")

        line_index += 1               #move to the next line after reading vertices and edges

        edges = []                     #list to store the edges for the current graph
        for __ in range(num_edges):
            #skip any empty lines before reading the edges
            while line_index < len(lines) and not lines[line_index].strip():
                line_index += 1

            if line_index >= len(lines):   #prevent going beyond the file content
                raise ValueError("Unexpected end of file. Not enough data for edges.")

            # Read each edge (start, end, weight) and append it to the edge list
            try:
                u, v, w = map(int, lines[line_index].strip().split())
                edges.append((u, v, w))
            except ValueError:
                raise ValueError(f"Malformed edge line: {lines[line_index]}")

            line_index += 1  #move to the next line after reading the edge

        graph_data.append((num_vertices, num_edges, edges))  #store the current graph data

    return num_graphs, graph_data 


if __name__ == "__main__":
    input_file_path = "rosalind_cte.txt" 

    num_graphs, graph_data = read_graph_input(input_file_path)

    cycle_lengths = find_shortest_cycle(num_graphs, graph_data)

    for length in cycle_lengths:
        print(length)

In [ ]:
#exercise_nwc

import networkx as nx

class GraphWithNegativeCycleCheck:
    def __init__(self):
        self.graph = nx.DiGraph()                #initialize an empty directed graph

    def build_graph(self, vertex_count, edges):
        #add a dummy node (-1) and connect it to all other nodes with weight 0
        self.graph.add_node(-1)  
        for vertex in range(1, vertex_count + 1):
            self.graph.add_node(vertex)
            self.graph.add_edge(-1, vertex, weight=0)  #add edges from -1 to all nodes
            
        #add all given edges to the graph
        for start, end, weight in edges:
            self.graph.add_edge(start, end, weight=weight)

    def contains_negative_cycle(self):
        try:
            #using Bellman-Ford algorithm to detect negative cycle
            nx.single_source_bellman_ford_path_length(self.graph, -1)
        except nx.NetworkXUnbounded:
            #if a negative cycle is detected, return "1"
            return "1"
        #if no negative cycle is found, return "-1"
        return "-1"

def process_input(input_file):
    with open(input_file, 'r') as file:
        lines = file.read().splitlines()
    
    graph_count = int(lines[0])  # read the number of graphs
    result_list = []             #list to store the results for each graph

    index = 1                    #start processing the graphs from the second line
    while index < len(lines):
        #skip empty lines that may exist between graph definitions
        if not lines[index].strip():
            index += 1
            continue
        
        #read the number of vertices and edges
        try:
            num_vertices, num_edges = map(int, lines[index].split())
        except ValueError:
            #skip malformed lines and continue processing
            print(f"Skipping malformed line: {lines[index]}")
            index += 1
            continue
        
        index += 1                   #move to the next line which contains edge information

        edge_list = []               #list to store edges for the current graph
        for _ in range(num_edges):
            #skip any empty lines in between edge data
            while index < len(lines) and not lines[index].strip():
                index += 1  # Skip empty lines

            if index >= len(lines):       #making sure that there are still lines to read
                break

            #read edge data and append to the edge list
            try:
                start, end, weight = map(int, lines[index].split())
                edge_list.append((start, end, weight))
            except ValueError:
                print(f"Skipping malformed edge line: {lines[index]}")
            index += 1  # Move to the next edge line
        
        #creating the graph and check for negative weight cycle
        graph = GraphWithNegativeCycleCheck()
        graph.build_graph(num_vertices, edge_list)
        result_list.append(graph.contains_negative_cycle())
    
    print(" ".join(result_list))

if __name__ == "__main__":
    process_input('rosalind_nwc.txt')


In [ ]:
#exercise_sdag

from collections import defaultdict

#function to perform a topological sort using DFS
def topological_sort(n, adj):
    visited = [False] * (n + 1)
    topo_sort = []

    def dfs(v):
        visited[v] = True
        for u, _ in adj[v]:
            if not visited[u]:
                dfs(u)
        topo_sort.append(v)

    for i in range(1, n + 1):
        if not visited[i]:
            dfs(i)

    topo_sort.reverse()
    return topo_sort

#function to find the shortest path in a DAG
def shortest_path_dag(n, edges):
    adj = defaultdict(list)
    for u, v, w in edges:
        adj[u].append((v, w))

    #perform topological sort
    topo_order = topological_sort(n, adj)

    #initialize distance array
    D = [float('inf')] * (n + 1)
    D[1] = 0  # Assuming source node is 1

    #relax edges in topological order
    for u in topo_order:
        if D[u] != float('inf'):
            for v, weight in adj[u]:
                if D[u] + weight < D[v]:
                    D[v] = D[u] + weight

    #prepare the result
    result = []
    for i in range(1, n + 1):
        if D[i] == float('inf'):
            result.append('x')
        else:
            result.append(str(D[i]))

    return result

#reading input and running the algorithm
def read_input(file_path):
    try:
        with open(file_path, 'r') as file:
            n, m = map(int, file.readline().split())
            edges = []
            for _ in range(m):
                u, v, w = map(int, file.readline().split())
                edges.append((u, v, w))
        return n, edges
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None, None
    except ValueError:
        print("Error: Invalid file format.")
        return None, None

def main():
    input_file = "rosalind_sdag.txt"
    n, edges = read_input(input_file)
    if n is None or edges is None:
        return

    if n == 0:
        print("x")
        return

    result = shortest_path_dag(n, edges)
    print(" ".join(result))

if __name__ == "__main__":
    main()